In [ ]:
!pip install fastai2
!pip install fastai2>=0.0.11 graphviz kornia ipywidgets matplotlib nbdev>=0.2.12 pandas scikit_learn azure-cognitiveservices-search-imagesearch sentencepiece

In [ ]:
from fastai2.vision.all import *
from fastai2.medical.imaging import *

In [ ]:
trn_path = Path('../input/rsna-hemorrhage-jpg/train_jpg/train_jpg/')
csv_path = Path('../input/all-csvs/')

In [ ]:
# fn = get_image_files(trn_path)
# len(fn)

In [ ]:
labels = pd.read_csv(csv_path/'new_labels.csv')
labels.head()

In [ ]:
labels.columns

In [ ]:
labels = labels.drop(['Unnamed: 0', 'epidural', 'intraparenchymal', 'intraventricular',
       'subarachnoid', 'subdural'], axis = 1)
labels.head()

We have two types of transforms available to us, item transforms and batch transforms. Item transforms are things we do before using the images, eg resizing all of them to the same size before feeding to the NN. Item transforms are done on the whole dataset and on cpu. Batch transforms on the other hand are done on the gpu, one batch at a time. They're things like random rotation which will help with training

In [ ]:
tfms = L([FlipItem(p = 0.2)])

In [ ]:
dls = ImageDataLoaders.from_df(labels, trn_path, label_delim=';', batch_tfms=tfms, seed = 42)

In [ ]:
dls.show_batch()

In [ ]:
len(dls.train_ds), len(dls.valid_ds), dls.bs

In [ ]:
learn = cnn_learner(dls, resnet50, metrics=partial(accuracy_multi, thresh=0.5), model_dir = '/kaggle')

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(5, base_lr= 3e-2)

In [ ]:
learn.save("/kaggle/working/multilabel-res50-stage-1")

In [ ]:
learn.show_results()

In [ ]:
learn.recorder.plot_loss()

In [ ]:
learn.lr_find()

In [ ]:
learn.fine_tune(3, base_lr = 9e-6)

In [ ]:
learn.save("/kaggle/working/multilabel-res50-stage-2")

In [ ]:
learn.show_results()